In [1]:
import pandas as pd
import numpy as np

In [2]:
loan_result = pd.read_csv("./data/loan_result.csv")
log_data = pd.read_csv("./data/log_data.csv")
user_spec = pd.read_csv("./data/user_spec.csv")

In [3]:
check_applied_ratio = loan_result[loan_result["loanapply_insert_time"] < "2022-06-01"]

In [5]:
check_applied_ratio["is_applied"].value_counts()

0.0    9709675
1.0     560449
Name: is_applied, dtype: int64

In [7]:
9709675/(9709675 + 560449)

0.9454291885862333

In [8]:
560449/(9709675 + 560449)

0.05457081141376677

In [4]:
kk = loan_result[loan_result["loanapply_insert_time"] >= "2022-06-01"]

In [5]:
kk[kk["application_id"] == 1882657]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
425,1882657,2022-06-07 11:09:57,13,123,0.0,20.0,NaN


In [6]:
#test = loan_result[loan_result["loanapply_insert_time"] >= "2022-06-01 00:00:00"]
#loan_result = loan_result[loan_result["loanapply_insert_time"] < "2022-06-01 00:00:00"]


### Step 1. Merge Dataset 

- 1. Drop columns : [personal_rehabilitation_yn , personal_rehabilitation_complete_yn ]
    - 위 컬럼들은 애초에 결측치가 너무 많고 추정하기가 쉽지않음.

In [7]:
loan_result.merge(user_spec).isna().sum()

application_id                                0
loanapply_insert_time                         0
bank_id                                       0
product_id                                    0
loan_limit                                 7382
loan_rate                                  7382
is_applied                              3257239
user_id                                       0
birth_year                               128096
gender                                   128096
insert_time                                   0
credit_score                            1509276
yearly_income                                 6
income_type                                   0
company_enter_month                      400337
employment_type                               0
houseown_type                                 0
desired_amount                                0
purpose                                       0
personal_rehabilitation_yn              5888701
personal_rehabilitation_complete_yn    1

In [8]:
#test = test.merge(user_spec)
loan_result = loan_result.merge(user_spec)

In [9]:
#test = test.drop(["personal_rehabilitation_yn" , "personal_rehabilitation_complete_yn" ], axis = 1)
loan_result = loan_result.drop(["personal_rehabilitation_yn" , "personal_rehabilitation_complete_yn" ], axis = 1)


### Step 2. Imputation

- loan_limit이 nan이면 loan_rate 역시 nan임
- 이 결측치들은 각 application_id의 평균값으로 대체

In [10]:
unique_loan = loan_result[loan_result['loan_limit'].isnull()]["application_id"].unique()

for i in range(len(unique_loan)):
    mean = loan_result[loan_result["application_id"] == unique_loan[i]].loc[:,"loan_limit"].dropna().mean()
    loan_result.loc[loan_result[loan_result["application_id"] == unique_loan[i]].loc[:,"loan_limit"].fillna( mean).index, "loan_limit"] = loan_result[loan_result["application_id"] == unique_loan[i]].loc[:,"loan_limit"].fillna( mean)

In [11]:
unique_loan = loan_result[loan_result['loan_rate'].isnull()]["application_id"].unique()

for i in range(len(unique_loan)):
    mean = loan_result[loan_result["application_id"] == unique_loan[i]].loc[:,"loan_rate"].dropna().mean()
    loan_result.loc[loan_result[loan_result["application_id"] == unique_loan[i]].loc[:,"loan_rate"].fillna( mean).index, "loan_rate"] = loan_result[loan_result["application_id"] == unique_loan[i]].loc[:,"loan_rate"].fillna( mean)

In [12]:
loan_result.isna().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                    26
loan_rate                     26
is_applied               3257239
user_id                        0
birth_year                128096
gender                    128096
insert_time                    0
credit_score             1509276
yearly_income                  6
income_type                    0
company_enter_month       400337
employment_type                0
houseown_type                  0
desired_amount                 0
purpose                        0
existing_loan_cnt        2685709
existing_loan_amt        3890163
dtype: int64

### 2-1 결측치 추정

#### 1.1 Birty_year & Gender 결측치 추정
- 기존에 있으면 이걸로 대체 (이건 전부 결측치임)

In [13]:
birth_data = loan_result[loan_result['birth_year'].isnull()]["application_id"].unique()

In [14]:
for i in range(len(birth_data)):
    mean = loan_result[loan_result["application_id"] == birth_data[i]].loc[:,"birth_year"].dropna().mean()
    loan_result.loc[loan_result[loan_result["application_id"] == birth_data[i]].loc[:,"birth_year"].fillna( mean).index, "birth_year"] = loan_result[loan_result["application_id"] == birth_data[i]].loc[:,"birth_year"].fillna(mean)

In [15]:
gender_data = loan_result[loan_result['gender'].isnull()]["application_id"].unique()

In [16]:
for i in range(len(gender_data)):
    mean = loan_result[loan_result["application_id"] == gender_data[i]].loc[:,"gender"].dropna().mean()
    loan_result.loc[loan_result[loan_result["application_id"] == gender_data[i]].loc[:,"gender"].fillna( mean).index, "gender"] = loan_result[loan_result["application_id"] == gender_data[i]].loc[:,"gender"].fillna(mean)

In [17]:
loan_result.isna().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                    26
loan_rate                     26
is_applied               3257239
user_id                        0
birth_year                128096
gender                    128096
insert_time                    0
credit_score             1509276
yearly_income                  6
income_type                    0
company_enter_month       400337
employment_type                0
houseown_type                  0
desired_amount                 0
purpose                        0
existing_loan_cnt        2685709
existing_loan_amt        3890163
dtype: int64

### 2-1 결측치 추정

#### 1.2 Credit_score 결측치 추정
- 기존에 있으면 이걸로 대체 
- 2022.06월 이후는 다 결측치인가본데..?

In [18]:
credit_data = loan_result[loan_result['credit_score'].isnull()]["application_id"].unique()

In [19]:
for i in range(len(credit_data)):
    mean = loan_result[loan_result["application_id"] == credit_data[i]].loc[:,"credit_score"].dropna().mean()
    loan_result.loc[loan_result[loan_result["application_id"] == credit_data[i]].loc[:,"credit_score"].fillna( mean).index, "credit_score"] = loan_result[loan_result["application_id"] == credit_data[i]].loc[:,"credit_score"].fillna(mean)

In [20]:
loan_result.isna().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                    26
loan_rate                     26
is_applied               3257239
user_id                        0
birth_year                128096
gender                    128096
insert_time                    0
credit_score             1509276
yearly_income                  6
income_type                    0
company_enter_month       400337
employment_type                0
houseown_type                  0
desired_amount                 0
purpose                        0
existing_loan_cnt        2685709
existing_loan_amt        3890163
dtype: int64

### 1.6 company_enter_month

- 근속년수를 파악할 수 있는 중요한 도구니깐.. 컬럼을 제거하면 안됨.
- 그런데 이걸 추정할 수 있는 마땅한 방법이 없음... 대체적으로 프리랜서인경우가 이거에 속할텐데 어떻게 해야할지 고민해야함
- company_enter_month 값들은 모두 0으로 바꾸고, 근속년수를 계산하기

In [326]:
loan_result_2 = pd.read_csv("./data/loan_result.csv")

In [327]:
loan_result_2 = loan_result_2.merge(user_spec)

In [328]:
loan_result["company_enter_month"] = loan_result_2["company_enter_month"]

In [329]:
# 최소 17살 이상일해야함
loan_result.loc[loan_result[((loan_result["birth_year"] * 100)+ 1700)>= loan_result["company_enter_month"] ].index, "company_enter_month"] = np.nan

In [330]:
loan_result.loc[loan_result[loan_result['company_enter_month'].isnull()].index, "company_enter_month"] = "000000"

In [331]:
loan_result['company_enter_month'] = loan_result['company_enter_month'].astype(float)

In [332]:
loan_result['company_enter_month'] = loan_result['company_enter_month'].astype(str).str[:6]

In [340]:
loan_result['company_enter_month'] = loan_result['company_enter_month'].astype(float).astype(int)

In [341]:
loan_result.loc[loan_result[loan_result['company_enter_month'] == 202211].index, "company_enter_month"] = 0

In [344]:
loan_result['company_enter_month'] = (202206 - loan_result['company_enter_month'])

In [370]:
348 / 12

29.0

In [371]:
def company_enter_enc(x):
    if x <= 3: return 0
    elif x <= 6: return 1
    elif x <= 9: return 2
    elif x <= 12: return 3
    elif x <= 24: return 4
    elif x <= 36: return 5
    elif x <= 48: return 6
    elif x <= 60: return 7
    elif x <= 72: return 8
    elif x <= 84: return 9
    elif x <= 96: return 10
    elif x <= 108: return 11
    elif x <= 120: return 12
    elif x <= 132: return 13
    elif x <= 144: return 14
    elif x <= 168: return 15
    elif x <= 180: return 16
    elif x <= 192: return 17
    elif x <= 204: return 18
    elif x <= 216: return 19
    elif x <= 228: return 20
    elif x <= 240: return 21
    elif x <= 252: return 22
    elif x <= 264: return 23
    elif x <= 276: return 24
    elif x <= 288: return 25
    elif x <= 300: return 26
    elif x <= 312: return 27
    elif x <= 324: return 28
    elif x <= 336: return 29
    elif x <= 348: return 30
    elif x <= 360: return 31

    else: return 99


In [373]:
loan_result["company_enter_month"] = loan_result["company_enter_month"].map(company_enter_enc)

### 1.8. existing_loan_cnt, existing_loan_amt

In [374]:
loan_result.loc[loan_result['existing_loan_cnt'] != loan_result['existing_loan_cnt'], 'existing_loan_cnt'] = 0

In [375]:
loan_result.loc[loan_result['existing_loan_amt'] != loan_result['existing_loan_amt'], 'existing_loan_amt'] = 0

In [376]:
loan_result.isna().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                    26
loan_rate                     26
is_applied               3257239
user_id                        0
birth_year                128096
gender                    128096
insert_time                    0
credit_score             1509276
yearly_income                  6
income_type                    0
company_enter_month            0
employment_type                0
houseown_type                  0
desired_amount                 0
purpose                        0
existing_loan_cnt              0
existing_loan_amt              0
dtype: int64

### 1.9 Credit Score 다시생각해보기 - 일단제거

In [377]:
loan_result.loc[loan_result['purpose'] == "SWITCHLOAN", 'purpose'] = "대환대출"
loan_result.loc[loan_result['purpose'] == "LIVING", 'purpose'] = "생활비"
loan_result.loc[loan_result['purpose'] == "ETC", 'purpose'] = "기타"
loan_result.loc[loan_result['purpose'] == "BUYCAR", 'purpose'] = "자동차구입"
loan_result.loc[loan_result['purpose'] == "BUSINESS", 'purpose'] = "사업자금"
loan_result.loc[loan_result['purpose'] == "HOUSEDEPOSIT", 'purpose'] = "전월세보증금"
loan_result.loc[loan_result['purpose'] == "INVEST", 'purpose'] = "투자"
loan_result.loc[loan_result['purpose'] == "BUYHOUSE", 'purpose'] = "주택구입"


In [378]:
loan_result["income_type"].unique()

array(['EARNEDINCOME', 'FREELANCER', 'PRIVATEBUSINESS', 'OTHERINCOME',
       'EARNEDINCOME2', 'PRACTITIONER'], dtype=object)

### Data Merge

In [379]:
test = loan_result[loan_result["loanapply_insert_time"] >= "2022-06-01 00:00:00"]
loan_result = loan_result[loan_result["loanapply_insert_time"] < "2022-06-01 00:00:00"]


In [380]:
test.to_csv("./prepro_data/submit_test.csv", index = False)

In [381]:
loan_result.to_csv("./prepro_data/full_data.csv", index = False)

In [382]:
test_set = pd.read_csv("./prepro_data/submit_test.csv")

In [383]:
test_set

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN,430982,1996.0,1.0,...,620.0,24000000.0,EARNEDINCOME,1,정규직,자가,25000000.0,대환대출,2.0,15000000.0
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN,430982,1996.0,1.0,...,620.0,24000000.0,EARNEDINCOME,1,정규직,자가,25000000.0,대환대출,2.0,15000000.0
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN,430982,1996.0,1.0,...,620.0,24000000.0,EARNEDINCOME,1,정규직,자가,25000000.0,대환대출,2.0,15000000.0
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN,430982,1996.0,1.0,...,620.0,24000000.0,EARNEDINCOME,1,정규직,자가,25000000.0,대환대출,2.0,15000000.0
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN,430982,1996.0,1.0,...,620.0,24000000.0,EARNEDINCOME,1,정규직,자가,25000000.0,대환대출,2.0,15000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257234,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN,478802,1965.0,1.0,...,610.0,45000000.0,EARNEDINCOME,27,정규직,기타가족소유,50000000.0,대환대출,2.0,56000000.0
3257235,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN,478802,1965.0,1.0,...,610.0,45000000.0,EARNEDINCOME,27,정규직,기타가족소유,50000000.0,대환대출,2.0,56000000.0
3257236,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN,478802,1965.0,1.0,...,610.0,45000000.0,EARNEDINCOME,27,정규직,기타가족소유,50000000.0,대환대출,2.0,56000000.0
3257237,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN,478802,1965.0,1.0,...,610.0,45000000.0,EARNEDINCOME,27,정규직,기타가족소유,50000000.0,대환대출,2.0,56000000.0


In [384]:
3257239 

3257239

### 고민해볼것들

In [431]:
k = loan_result.groupby("product_id").mean()["is_applied"] * 100
k = pd.DataFrame(k).reset_index().sort_values("is_applied")

In [432]:
kk = loan_result.groupby("product_id").median()["is_applied"] * 100
kk = pd.DataFrame(kk).reset_index().sort_values("is_applied")

In [433]:
k[k["is_applied"] >= 10]

,product_id,is_applied
156,244,10.031596
151,236,10.475341
91,146,10.659898
153,239,10.761485
122,191,11.403075
115,184,11.499675
129,200,11.652773
114,183,12.030799
169,261,12.087518
20,38,12.149275


In [419]:
k = loan_result.groupby("bank_id").mean()["is_applied"] * 100
k = pd.DataFrame(k).reset_index().sort_values("is_applied")

In [423]:
k[k["is_applied"] >= 10]

,bank_id,is_applied
7,8,10.100610
44,45,10.236079
6,7,10.303225
56,59,10.405889
47,49,10.774591
57,60,11.286055
3,4,19.958191
28,29,22.120345
15,16,80.000000


In [429]:
loan_result[loan_result["product_id"] == 244]

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,existing_loan_cnt,existing_loan_amt
27515,415899,2022-05-09 11:26:04,31,244,267000000.0,9.5,1.0,720368,1975.0,1.0,...,670.0,95000000.0,PRIVATEBUSINESS,99,정규직,자가,70000000.0,생활비,0.0,0.0
127528,932156,2022-05-12 01:39:20,31,244,200000000.0,8.6,0.0,532346,1983.0,1.0,...,710.0,45000000.0,PRIVATEBUSINESS,99,정규직,자가,50000000.0,대환대출,3.0,48000000.0
193441,1900377,2022-05-27 09:40:05,31,244,259000000.0,8.7,0.0,100422,1973.0,1.0,...,940.0,50000000.0,PRIVATEBUSINESS,99,정규직,자가,100000000.0,사업자금,0.0,0.0
281203,898557,2022-04-02 11:07:35,31,244,284000000.0,9.1,0.0,688010,1987.0,0.0,...,570.0,59000000.0,PRIVATEBUSINESS,99,정규직,자가,10000000.0,생활비,6.0,50000000.0
285546,1394080,2022-04-02 02:46:00,31,244,33000000.0,8.6,0.0,689761,1976.0,1.0,...,680.0,90000000.0,PRIVATEBUSINESS,18,정규직,자가,20000000.0,사업자금,14.0,399000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13305767,2062418,2022-03-08 10:01:19,31,244,126000000.0,8.3,0.0,564397,1986.0,1.0,...,910.0,96000000.0,EARNEDINCOME,99,정규직,자가,180000000.0,대환대출,8.0,496000000.0
13307244,435763,2022-03-08 10:41:08,31,244,512000000.0,8.2,0.0,811273,1984.0,1.0,...,820.0,43000000.0,EARNEDINCOME,19,정규직,자가,100000000.0,대환대출,4.0,307000000.0
13434856,2079783,2022-05-16 13:39:31,31,244,192000000.0,8.4,0.0,827523,1980.0,1.0,...,870.0,0.0,PRIVATEBUSINESS,99,정규직,자가,10000000.0,생활비,6.0,171000000.0
13460537,1287414,2022-05-16 00:32:02,31,244,249000000.0,8.7,0.0,100422,1973.0,1.0,...,940.0,50000000.0,PRIVATEBUSINESS,99,정규직,기타가족소유,100000000.0,기타,0.0,0.0


In [457]:
k = loan_result.groupby("existing_loan_cnt").mean()["is_applied"]

In [458]:
k = pd.DataFrame(k).reset_index().sort_values("is_applied")

In [466]:
k[k["existing_loan_cnt"] > 4]["is_applied"].mean()

0.2138007432396386

In [461]:
loan_result["existing_loan_cnt"].describe()

count    1.027001e+07
mean     2.884593e+00
std      3.166278e+00
min      0.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      1.780000e+02
Name: existing_loan_cnt, dtype: float64

In [467]:
loan_result.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'birth_year',
       'gender', 'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'company_enter_month', 'employment_type', 'houseown_type',
       'desired_amount', 'purpose', 'existing_loan_cnt', 'existing_loan_amt'],
      dtype='object')

In [470]:
check_the_set_cover = loan_result[["application_id", "loanapply_insert_time", 'loan_limit', 'loan_rate', 'is_applied']]

2157865

In [487]:
check_the_set_cover[check_the_set_cover["application_id"] == check_the_set_cover["application_id"].unique()[13]]

,application_id,loanapply_insert_time,loan_limit,loan_rate,is_applied
13549,1637434,2022-05-09 17:33:24,7000000.0,4.0,0.0
13550,1637434,2022-05-09 17:33:37,39000000.0,11.3,0.0
13551,1637434,2022-05-09 17:33:26,49000000.0,12.9,0.0
13552,1637434,2022-05-09 17:33:25,30000000.0,3.9,1.0
13553,1637434,2022-05-09 17:33:26,27000000.0,13.0,0.0
13554,1637434,2022-05-09 17:33:25,14000000.0,7.5,0.0
13555,1637434,2022-05-09 17:33:38,30000000.0,17.1,0.0
13556,1637434,2022-05-09 17:33:25,19000000.0,13.3,0.0
13557,1637434,2022-05-09 17:33:26,25000000.0,11.6,0.0
13558,1637434,2022-05-09 17:33:25,31000000.0,11.5,0.0
